In [2]:
# Install required libraries if not already present
!pip install azure-storage-blob pandas numpy

from azure.storage.blob import BlobServiceClient
import pandas as pd
from datetime import datetime
import io
import numpy as np

# Connect to Azure Blob Storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=sg092620240215;AccountKey=+PaTF6WCZ0NY63Hni1XIWRJfWsnTI7QJCLVP0f1OXUoVzJyl0AcE4h2Pe1b7ZbgldGkDDFA0j9iK+AStvU4auA==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client("silver")

# Function to find the latest merged_rosters_players_with_groups_*.csv
def get_latest_csv(container_client, prefix="merged_rosters_players_with_groups_"):
    latest_blob = None
    latest_time = None
    for blob in container_client.list_blobs(name_starts_with=prefix):
        print(f"Processing blob: {blob.name}")
        try:
            parts = blob.name.split("_")
            if len(parts) < 6:
                print(f"Skipping {blob.name}: Not enough parts")
                continue
            timestamp_str = f"{parts[-2]}_{parts[-1].replace('.csv', '')}"
            timestamp = datetime.strptime(timestamp_str, "%Y%m%d_%H%M%S")
            if latest_time is None or timestamp > latest_time:
                latest_time = timestamp
                latest_blob = blob.name
        except ValueError as e:
            print(f"Skipping {blob.name}: Invalid timestamp ({e})")
            continue
    return latest_blob

# Check for latest CSV
latest_blob = get_latest_csv(container_client)
if not latest_blob:
    raise ValueError("No merged_rosters_players_with_groups_*.csv files found")
else:
    print(f"Latest CSV found: {latest_blob}")

# Load the latest CSV
print(f"Loading {latest_blob}")
blob_client = container_client.get_blob_client(latest_blob)
blob_data = blob_client.download_blob().readall().decode("utf-8")
df = pd.read_csv(io.StringIO(blob_data))

# Normalize ADP and salary by position_group for year=2025 (min-max)
df_2025 = df[df['year'] == 2025].copy()
for group in df_2025['position_group'].unique():
    mask = df_2025['position_group'] == group
    # Normalize ADP
    adp_min = df_2025.loc[mask, 'ADP'].min()
    adp_max = df_2025.loc[mask, 'ADP'].max()
    if adp_max > adp_min:
        df_2025.loc[mask, 'ADP_normalized'] = (df_2025.loc[mask, 'ADP'] - adp_min) / (adp_max - adp_min)
    else:
        df_2025.loc[mask, 'ADP_normalized'] = 0.0
    
    # Normalize salary
    salary_min = df_2025.loc[mask, 'salary'].min()
    salary_max = df_2025.loc[mask, 'salary'].max()
    if salary_max > salary_min:
        df_2025.loc[mask, 'salary_normalized_by_pos'] = (df_2025.loc[mask, 'salary'] - salary_min) / (salary_max - salary_min)
    else:
        df_2025.loc[mask, 'salary_normalized_by_pos'] = 0.0

# Merge normalized columns back
df = df.merge(df_2025[['id', 'ADP_normalized', 'salary_normalized_by_pos']], on='id', how='left')
df['ADP_normalized'] = df['ADP_normalized'].fillna(0.0)
df['salary_normalized_by_pos'] = df['salary_normalized_by_pos'].fillna(0.0)

# Normalize salary by position_group and teamName for all years (min-max)
for pos_group, team_group in df.groupby(['position_group', 'teamName']).groups.keys():
    mask = (df['position_group'] == pos_group) & (df['teamName'] == team_group)
    salary_min = df.loc[mask, 'salary'].min()
    salary_max = df.loc[mask, 'salary'].max()
    if salary_max > salary_min:
        df.loc[mask, 'salary_normalized_by_pos_team'] = (df.loc[mask, 'salary'] - salary_min) / (salary_max - salary_min)
    else:
        df.loc[mask, 'salary_normalized_by_pos_team'] = 0.0

# Z-score and index for 2025 by position_group
df_2025 = df[df['year'] == 2025].copy()
for group in df_2025['position_group'].unique():
    mask = df_2025['position_group'] == group
    # Z-score ADP (negated so lower is better)
    adp_mean = df_2025.loc[mask, 'ADP'].mean()
    adp_std = df_2025.loc[mask, 'ADP'].std()
    if adp_std > 0:
        df_2025.loc[mask, 'ADP_zscore'] = -1 * (df_2025.loc[mask, 'ADP'] - adp_mean) / adp_std
    else:
        df_2025.loc[mask, 'ADP_zscore'] = 0.0
    
    # Z-score salary (negated so lower is better)
    salary_mean = df_2025.loc[mask, 'salary'].mean()
    salary_std = df_2025.loc[mask, 'salary'].std()
    if salary_std > 0:
        df_2025.loc[mask, 'salary_zscore_by_pos'] = -1 * (df_2025.loc[mask, 'salary'] - salary_mean) / salary_std
    else:
        df_2025.loc[mask, 'salary_zscore_by_pos'] = 0.0
    
    # Index: Average of negated z-scores (higher = better value)
    df_2025.loc[mask, 'value_index_by_pos'] = (df_2025.loc[mask, 'ADP_zscore'] + df_2025.loc[mask, 'salary_zscore_by_pos']) / 2

# Merge z-scores and index back
df = df.merge(df_2025[['id', 'ADP_zscore', 'salary_zscore_by_pos', 'value_index_by_pos']], on='id', how='left')
df[['ADP_zscore', 'salary_zscore_by_pos', 'value_index_by_pos']] = df[['ADP_zscore', 'salary_zscore_by_pos', 'value_index_by_pos']].fillna(0.0)

# Z-score and index for all years by position_group and teamName
for pos_group, team_group in df.groupby(['position_group', 'teamName']).groups.keys():
    mask = (df['position_group'] == pos_group) & (df['teamName'] == team_group)
    # Z-score salary (negated)
    salary_mean = df.loc[mask, 'salary'].mean()
    salary_std = df.loc[mask, 'salary'].std()
    if salary_std > 0:
        df.loc[mask, 'salary_zscore_by_pos_team'] = -1 * (df.loc[mask, 'salary'] - salary_mean) / salary_std
    else:
        df.loc[mask, 'salary_zscore_by_pos_team'] = 0.0

# Save the updated DataFrame back to silver
output_blob_name = f"merged_rosters_players_normalized_and_indexed_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}.csv"
output_blob_client = container_client.get_blob_client(output_blob_name)
output_csv = df.to_csv(index=False)
output_blob_client.upload_blob(output_csv, overwrite=True)
print(f"Uploaded normalized and indexed CSV to silver as {output_blob_name}")

# Display the updated DataFrame
print("Updated DataFrame (first 5 rows):")
print(df.head())

StatementMeta(b73298e2-f1b8-4875-a47a-e543f5595c3b, 6, 7, Finished, Available, Finished)

Processing blob: merged_rosters_players_with_groups_20250308_155958.csv
Latest CSV found: merged_rosters_players_with_groups_20250308_155958.csv
Loading merged_rosters_players_with_groups_20250308_155958.csv
Uploaded normalized and indexed CSV to silver as merged_rosters_players_normalized_and_indexed_20250309_005532.csv
Updated DataFrame (first 5 rows):
  pos              name     id   ADP  year               teamName position  \
0  DH    Ohtani, Shohei  02yc4  1.34  2022     Cleveland Steamers        P   
1  DH    Ohtani, Shohei  02yc4  1.34  2023    Steel City Baseball      OFD   
2  DH    Ohtani, Shohei  02yc4  1.34  2021     Cleveland Steamers      OFD   
3  DH  Ohtani-H, Shohei  06alt  1.91  2024              SnowBlind      OFD   
4  DH  Ohtani-H, Shohei  06alt  1.91  2025  Excuse me, I?m Olerud      OFD   

       salary salary_display position_group  ADP_normalized  \
0  22000000.0           $22M            OFD             0.0   
1  50000000.0           $50M            OFD     